In [ ]:
import os
import sys
from pathlib import Path

PROJ_ROOT = Path.cwd().parent
if str(PROJ_ROOT) not in sys.path:
    sys.path.append(str(PROJ_ROOT))

import pandas as pd
import numpy as np
import pydicom
pydicom.config.settings.reading_validation_mode = pydicom.config.IGNORE

import SimpleITK as sitk

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from ipywidgets import interact

from utils.functions import load_config
from dataset.common import load_patient_data
from dataset.process import GeneratePatientData

In [ ]:
cfg = load_config()

# Make Patient Image-XML Matching Table

## Dicom Files

### Generated Strucutred Data from DICOM

In [ ]:
# Run this cell to generate patient data
# patient_df = GeneratePatientData(cfg.paths.raw_data, saveFileName='patient_data_w_metadata.csv')

In [ ]:
# Load Patient Data
patient_df = load_patient_data(cfg.paths.processed_data+'patient_data_w_metadata.csv')
patient_df

### Example - CT Images

In [ ]:
temp_images

In [ ]:
def browse_images(idx):
    plt.figure(figsize=(5, 5))
    plt.imshow(temp[idx].pixel_array, cmap='gray')
    plt.title(f"Instance: {temp[idx].InstanceNumber} || Total: {len(temp)}")
    plt.axis('off')
    plt.show()

In [ ]:
interact(browse_images, idx=(0, len(temp)-1))

## XML Files

In [ ]:
xml_path = os.path.join(cfg.paths.raw_data, 'calcium_xml')
temp_xml_path = os.path.join(xml_path, '1.xml')
temp_xml = load_single_xml(temp_xml_path)

In [ ]:
temp_xml[0]